In [3]:
from normality import normalize
import pandas as pd
from thefuzz import fuzz
from thefuzz import process
from countrynames import to_code
import os
import shutil
import datetime


NameError: name 'countrynames' is not defined

In [2]:

csv_to_fix = "ActivatedCarbon_MVPVersion01_EF_20230621.csv"


now = datetime.datetime.now()
formatted_date = now.strftime("%Y%m%d")
folder_path = "%s_Country_Renamed_%s" %(csv_to_fix.split("_")[0], formatted_date)
os.makedirs(folder_path)

files_to_keep = [csv_to_fix, 'ISOToEcoInventToolSub/Geographies.csv']
def delete_csv_files(directory):
    files = os.listdir(directory)
    for file in files:
        if file.endswith('.csv') and file not in files_to_keep:
            file_path = os.path.join(directory, file)
            os.remove(file_path)
            print(f"Deleted: {file_path}")
geo_df = pd.read_csv("./ISOToEcoInventToolSub/Geographies.csv")
df = pd.read_csv(csv_to_fix)



FileNotFoundError: [Errno 2] No such file or directory: './ISOToEcoInventToolSub/Geographies.csv'

In [8]:
df['Country of Applicability'].unique()
df['Country of Applicability'] = 'GLO'
df.to_csv("./%s/%s.csv"  %(folder_path, folder_path), index=False)

In [25]:
def normalize_name(country):
    """Clean up a country name before comparison."""
    return normalize(country, latinize=True)

df_ambig = pd.DataFrame(columns=["Input string", "Mapped name", "Mapped code", "Quarantine?"])

def mapping(country:str)->str:
    global df_ambig
    if(result:=to_code(country)) is not None:
        return result
    matching_rows = df_ambig[df_ambig["Input string"].str.contains(country, case=False, na=False)]
    if not matching_rows.empty:
        return country
    else:
        longname = process.extractOne(country, choices=geo_df['Name'], processor=(lambda x: str(x)), scorer=fuzz.token_set_ratio, score_cutoff=0)[0]
        shortname = geo_df[geo_df['Name'] == longname]['Shortname'].iloc[0]
        new_row = pd.DataFrame([{ 
            "Input string": country,
            "Mapped name": longname,
            "Mapped code": shortname}])
        df_ambig = pd.concat([df_ambig, new_row], ignore_index=True)
        return country
    
Region_column_name = 'Country of Applicability'
df[Region_column_name] = df[Region_column_name].apply(normalize_name)
df[Region_column_name] =df[Region_column_name].apply(lambda x: mapping(x))

df_ambig.to_csv("ambig.csv", index=False, sep='|')
df_ambig.to_csv("./%s/AMBIG_%s.csv" %(folder_path, folder_path), index=False)


In [26]:
input("Go to ambig.csv and correct ambiguous inputs")

''

In [30]:
def split_ambig():
    ambig = pd.read_csv('./ambig.csv')
    if ambig.empty:
        return ambig, ambig
    corrected = ambig[ambig['Quarantine?'] != 'x']
    further_quarantined = ambig[ambig['Quarantine?'] == 'x']
    return corrected, further_quarantined

def remove_quarantined(df_first_clean, further_quarantined, corrected):
    overlap_values = further_quarantined['Input string'].unique()
    removed_rows_df = df_first_clean[df_first_clean['Country of Applicability'].isin(overlap_values)].copy()
    df_second_clean = df_first_clean[~df_first_clean['Country of Applicability'].isin(overlap_values)]
    
    merged_df = pd.merge(df_second_clean, corrected, left_on='Country of Applicability', right_on='Input string', how='left')
    merged_df.loc[merged_df['Input string'].notna(), 'Country of Applicability'] = merged_df['Mapped code']
    merged_df.drop(columns=['Mapped code', "Input string","Mapped name","Quarantine?"], inplace=True)
    
    file_path = "./ambig.csv"
    if os.path.exists(file_path):
        shutil.move(file_path, "./%s/ambig_VERIFIED_%s.csv" % (folder_path, folder_path))
    return merged_df, removed_rows_df



corrected, further_quarantined = split_ambig()
merged_df, removed_rows_df =remove_quarantined(df, further_quarantined, corrected)




KeyError: 'Input string'

In [21]:
# In this cell, please add any code you'd like to edit 
merged_df["Country of Applicability"].unique()
removed_rows_df.head(15)
rows_to_move = merged_df[merged_df['Country of Applicability'] == 'annex i countries']
removed_rows_df = pd.concat([removed_rows_df, rows_to_move], ignore_index=True)
merged_df = merged_df[merged_df['Country of Applicability'] != 'annex i countries']
merged_df.loc[merged_df['Country of Applicability'] == 'asia', 'Country of Applicability'] = 'RAS'
merged_df.loc[merged_df['Country of Applicability'] == 'africa', 'Country of Applicability'] = 'RAF'
merged_df['Country of Applicability'].unique()


array(['CSHH', 'KG', 'EE', 'DJ', 'UN-SAFRICA', 'LS', 'KR', 'RER', 'NP',
       'GR', 'RO', 'UN-SAMERICA', 'SZ', 'BZ', 'PE', 'BW', 'US', 'MW',
       'TW', 'DM', 'IR', 'BO', 'ZW', 'AO', 'ET', 'LR', 'CO', 'ZM', 'AT',
       'SD', 'TG', 'MA', 'VC', 'BF', 'SAS', 'GH', 'BE/LU', 'UN-EASIA',
       'UN-SEUROPE', 'MS', 'UN-NAFRICA', 'RAS', 'WEU', 'IE', 'IQ', 'MX',
       'FM', 'GW', 'AU', 'NO', 'UN-AUSTRALIANZ', 'EU', 'RNA',
       'UN-AMERICAS', 'UN-MELANESIA', 'LY', 'ES', 'CN', 'PY', 'UN-WASIA',
       'RU', 'DK', 'PH', 'BJ', 'LB', 'LT', 'IT', 'AR', 'PA', 'YE', 'KP',
       'NZ', 'LK', 'BB', 'GU', 'ZA', 'MZ', 'HN', 'GB', 'FR', 'HU', 'DZ',
       'UN-MAFRICA', 'KE', 'TD', 'MG', 'ML', 'GT', 'NE', 'GM', 'SK', 'CL',
       'UN-EAFRICA', 'UZ', 'NG', 'UN-NEUROPE', 'LA', 'VE', 'UN-SEASIA',
       'OM', 'IN', 'UY', 'MR', 'GY', 'AL', 'RS', 'PK', 'CSXX', 'CD', 'PS',
       'MN', 'KZ', 'CV', 'JM', 'CY', 'VN', 'ST', 'BG', 'PL', 'HT', 'CF',
       'GE', 'MD', 'IL', 'Central Asia', 'SB', 'UN-POLYNESIA', '

In [22]:
input("Look through each dataset and adjust any inconsistent values")

removed_rows_df.to_csv("./%s/%s_further_quarantined.csv" %(folder_path, folder_path), index=False) 
merged_df.to_csv("./%s/%s.csv"  %(folder_path, folder_path), index=False)



'CropResidueEmissions_Country_Renamed_20240207/CropResidueEmissions_MVPVersion01_EF_20230621.csv'

In [23]:
shutil.copy("./Nationalizer.ipynb", "%s" % folder_path)
shutil.move(csv_to_fix, "%s" % folder_path)
shutil.make_archive(folder_path.split('.')[0], 'zip', folder_path)
shutil.rmtree(folder_path)
